<a href="https://colab.research.google.com/github/YeongseoYoo/Baco_Project/blob/main/classify_bikeroad.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
#Setting Environment
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-xl8ychef/kobert-tokenizer_fccff5d452ab4acc8b7b629ad2f21ea2
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-xl8ychef/kobert-tokenizer_fccff5d452ab4acc8b7b629ad2f21ea2
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [31]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel

In [52]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
import pandas as pd
from tqdm import tqdm, tqdm_notebook

In [33]:
#GPU 사용 위해 import
import torchvision.datasets as dsets

In [34]:
#torch cuda를 사용할 수 있는지 확인하게 위한 작업
USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)

True


In [35]:
#Colab의 GPU 활용
device = torch.device('cuda:0' if USE_CUDA else 'cpu')
print(device)

cuda:0


In [36]:
# [AI Hub] 감정 분류를 위한 대화 음성 데이터셋
data = pd.read_csv('5차년도_2차.csv', encoding='cp949')

In [37]:
#기존의 7개의 감정을 자전거 추천 코스 카테고리로 묶어주기
#inplace=True는 변경 내용 원래 DataFrame에 바로 적용하기
data['상황'].replace(['fear', 'angry', 'sadness', 'disgust'], 0, inplace=True) #비추 코스
data['상황'].replace(['surprise', 'neutral'], 1, inplace=True) #무난 코스
data['상황'].replace('happiness', 2, inplace=True) #추천코스

#발화문,상황 으로 구성된 list 만들기
data_list = []
for ques, label in zip(data['발화문'], data['상황'])  :
    data = []
    data.append(ques)
    data.append(str(label))

    data_list.append(data)

In [38]:
#데이터 확인하기
print(data)
print(data_list[:10])

['걱정해줘서 고마워.', '0']
[['헐! 나 이벤트에 당첨 됐어.', '2'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '2'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '2'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '2'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '1'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '1'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '2'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '2'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '2'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '2']]


In [39]:
#train set과 test set으로 나누기
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [40]:
#개수 확인하기
print(len(dataset_train), len(dataset_test))

15499 3875


In [41]:
#데이터셋 토큰화
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [42]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [43]:
#파라미터 설정
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [44]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [45]:
#이제 BERTDataset 함수에 sklearn을 통해 train & test으로 나눈 데이터를 입력합
tok= tokenizer
data_train = BERTDataset(dataset_train, 0, 1, tok , vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok , vocab, max_len, True, False)

In [46]:
# torch 형식의 dataset을 만들어 입력 데이터셋의 전처리 마무리
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size = batch_size, num_workers = 5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size = batch_size, num_workers = 5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [47]:
# KoBERT 오픈소스 내 예제코드 : https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 7,   # 감정 클래스 수로 조정
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)


In [48]:
# BERT  모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [49]:
# optimizer와 schedule 설정
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [50]:
# calc_accuracy : 정확도 측정을 위한 함수
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

train_dataloader


In [53]:
#Train
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-53-027929dd73e5>:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0002238750457764 train acc 0.15625
epoch 1 batch id 201 loss 0.39233556389808655 train acc 0.7139303482587065
epoch 1 train acc 0.7495498971193416


<ipython-input-53-027929dd73e5>:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9519320843091335


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.0058821262791752815 train acc 1.0
epoch 5 batch id 201 loss 0.024542726576328278 train acc 0.992304104477612
epoch 5 train acc 0.9917695473251029


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9524443793911007


In [54]:
# predict : 학습 모델을 활용하여 다중 분류된 클래스를 출력해주는 함수

def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")


In [59]:
#mysql 연결
!pip install mysql-connector-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.8/409.8 kB 35.6 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3


In [60]:
import mysql.connector

# 데이터베이스 연결 정보 설정
db_config = {
    'host': 'svc.sel4.cloudtype.app',
    'port': '32438',
    'user': 'root',
    'password': '0000',
    'database': 'baco_db'
}

# 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()

In [61]:
sql = "SHOW COLUMNS FROM review;"
cursor.execute(sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
  print(row)

('review_id', 'bigint(20)', 'NO', 'PRI', None, 'auto_increment')
('content', 'varchar(255)', 'YES', '', None, '')
('date', 'date', 'YES', '', None, '')
('end_place', 'varchar(255)', 'YES', '', None, '')
('hashtag', 'varchar(255)', 'YES', '', None, '')
('start_place', 'varchar(255)', 'YES', '', None, '')
('member_id', 'bigint(20)', 'YES', 'MUL', None, '')
('route_point', 'text', 'YES', '', None, '')
('analyzed', 'int(11)', 'YES', '', None, '')


In [62]:
sql = "SELECT content FROM review"
cursor.execute(sql)

# 결과 가져오기
result = cursor.fetchall()

# 결과 출력
for row in result:
  print(row)

('후기 내용 텍스트 부분',)
('후기내용텍스트',)
('후기내용텍스트',)
('후기내용텍스트',)
('월드컵 경기장에서 석촌호수까지!',)
('서울시청에서 키에리까지!',)
('청계천 경복궁까지!',)
('어린이대공원부터 남산타워까지!',)
('홍대 여의도 한강공원까지!',)
('서울숲부터 월드컵 경기장까지!',)
('자전거 도로가 잘 구성되어있어요. 뷰가 너무 좋아요!',)
('산뷰는 너무 좋지만, 오르막이 너무 많이 있어서 너무 힘들어요. 올라갈 때도 힘들지만, 내려올 때는 급경사라 위험해요',)
('주변에 맛집도 많고 뷰가 너무 좋아요!',)
('청계천부터 숙대까지!',)
('주변에 맛집도 많고 산책로도 잘 되어있어요~!!!',)
('와우와우와ㅜ',)
('다시 한번더 테스트 ~!!!!!!!!1',)
('다시한번 더 테스트테스트',)
('ㄹㄹㄹㄹㄹㄹㄹㄹㄹㄹㄹㄹㄹㄹ',)
('ㄹㄹㄹㄹ',)
('ㄹㄹㄹㄹㄹㄹㄹㄹ 후기작성 나오는지 보여주는 테스틀ㄹㄹㄹㄹㄹㄹㄹㄹㄹ',)
('배포한 앱에서 테스트 실시',)


In [63]:
import mysql.connector
import time
# 데이터베이스 연결 정보 설정
db_config = {
    'host': 'svc.sel4.cloudtype.app',
    'port': '32438',
    'user': 'root',
    'password': '0000',
    'database': 'baco_db'
}

# 데이터베이스 연결
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()


# 실시간으로 추가되는 데이터를 가져오고 분석하여 저장
def process_realtime_data():
    while True:
        # 새로운 데이터 가져오기
        new_data_query = "SELECT content FROM review WHERE analyzed = 0"
        cursor.execute(new_data_query)
        new_data = cursor.fetchall()

        if new_data:
            for row in new_data:
                text_to_analyze = row[0]
                print("입력 문장:", text_to_analyze)

                # 분석 결과 예측
                analyze_text = predict(text_to_analyze)
                print("분석 결과:", analyze_text)

                # 분석 결과 데이터베이스에 저장
                update_query = f"UPDATE review SET hashtag = '{analyze_text}', analyzed = 1 WHERE content = '{text_to_analyze}';"
                cursor.execute(update_query)
                conn.commit()

        else:
            print("새로운 데이터 없음")
            # 적절한 대기 시간을 두고 반복 수행
            time.sleep(5)

try:
    process_realtime_data()
except KeyboardInterrupt:
    print("실시간 데이터 처리를 종료합니다.")

# 연결 닫기
cursor.close()
conn.close()

새로운 데이터 없음
새로운 데이터 없음
새로운 데이터 없음
새로운 데이터 없음
새로운 데이터 없음
실시간 데이터 처리를 종료합니다.
